# Masked Language Modeling

This notebook describes how one can pre-train their own AntiBERTa model using the HuggingFace framework. As a demo, we've included the tokenizer we've used, and 1% of the sequences that we used in our training, validation, and test sets of the paper.

## Setup

After running this shell code, you need to restart the session, and you do not need to run this one again.

In [8]:
!pip install datasets -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd "/content/drive/MyDrive/cibi_prj"

/content/drive/MyDrive/cibi_prj


In [3]:
ls ##list file in the cibi_prj folder to make you have dataset in zip file

all_structures/     pair.zip                            test3/
all_structures.zip  paratope-prediction-task_0/         test4/
antiberta/          paratope-prediction-task_0_2/       test5/
AVIDa-SARS-CoV-2/   paratope-prediction-task_42/        test.txt
create_dataset.py   paratope-prediction-v2-task_0/      tmp_trainer/
data/               paratope-prediction-v2-task_42/     train.py
data2/              paratope-predictionv2-task_42/      train.txt
data3/              paratope-prediction-v2-task_42_v2/  unpair/
dataset.py          __pycache__/                        unpaired_v2/
data.zip            saved/                              unpaired_v2.zip
E-GEOD-35489.zip    src/                                unpair.zip
model/              test/                               val.txt
models.py           test2/                              vocab.txt


In [ ]:
!unzip "/content/drive/MyDrive/cibi_prj/unpaired_v2.zip"

Archive:  /content/drive/MyDrive/cibi_prj/unpaired_v2.zip
replace unpaired/SRR13082934_1_Heavy_IGHD.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: unpaired/SRR13082934_1_Heavy_IGHD.csv  
  inflating: unpaired/SRR3106519_Heavy_Bulk.csv  
  inflating: unpaired/SRR13082934_1_Heavy_IGHE.csv  
  inflating: unpaired/SRR3106520_Heavy_Bulk.csv  
  inflating: unpaired/SRR13082934_1_Heavy_IGHG.csv  
  inflating: unpaired/SRR13082934_1_Heavy_IGHM.csv  
  inflating: unpaired/SRR13082935_1_Heavy_Bulk.csv  
  inflating: unpaired/SRR3106521_Heavy_Bulk.csv  
  inflating: unpaired/SRR13082935_1_Heavy_IGHA.csv  
  inflating: unpaired/SRR13082935_1_Heavy_IGHD.csv  
  inflating: unpaired/SRR3106522_Heavy_Bulk.csv  
  inflating: unpaired/SRR13082935_1_Heavy_IGHE.csv  
  inflating: unpaired/SRR13082935_1_Heavy_IGHG.csv  
  inflating: unpaired/SRR13082935_1_Heavy_IGHM.csv  
  inflating: unpaired/SRR13082936_1_Heavy_Bulk.csv  
  inflating: unpaired/SRR3106471_Heavy_Bulk.csv  
  inflating: unpaired/

In [3]:
from transformers import (
    RobertaConfig,
    RobertaTokenizer,
    RobertaForMaskedLM,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer,
)
from datasets import load_dataset
import os
import pandas as pd

In [4]:
# # Initialise the tokeniser
tokenizer = RobertaTokenizer.from_pretrained(
    "./antiberta/antibody-tokenizer"
)

# Initialise the data collator, which is necessary for batching
collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'RobertaTokenizer'.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Text Data preprocessing

In [5]:
# Read file CSV
df = pd.read_csv('./data3/unpair/ERR220397_Heavy_Bulk.csv', sep=',', skiprows=1, encoding='utf-8')

print(df.columns)
print(df.info)
print(df['sequence'])

0       NNCCCTATCCCCCTGTGTGCCTTGAGAGACGCCGACCACTCATCTC...
1       CTTCGGAGACCCTGTCCCTCACCTGCGCTGTCTATGGTGGGTCCTT...
2       NCCCCTATCCCCCTGTGTGCCTTGAGAGACGCCGACCACTCATCTC...
3       CGGGGAGTCTCTGAAGATCTCCTGTAAGGGTTCTGGATACAGCTTT...
4       NNNNCCTACCTCCCCTCGTGTGCCTTGAGAGACGCCGACCACTCAT...
                              ...                        
1767    CTTCGGAGACCCTGTCCCTCACCTGCGCTGTCTATGGTGGGTCCTT...
1768    CCCCATCCCCCTGTNTGCCCTTGAGNACGCCGACCACTCATCTCGT...
1769    CCTATCCCCTGTGTGCCTTAAGAGACGCCGCCACTCATCTCGTGAA...
1770    CTTCGGAGACCCTGTCCCTCACCTGCGCTGTCTATGGTGGGTCCTT...
1771    CTTCGAGACCCTGTCCCTCACCTGCGCTGTCTATGGTGGGTCCTTC...
Name: sequence, Length: 1772, dtype: object


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import glob

# 1. Read and combine all CSV files
all_sequences = []  # Store only sequences directly

for file in glob.glob('./unpaired_v2/unpaired/*.csv'):  # Adjust the path if needed
    try:
        df = pd.read_csv(file, sep=',', skiprows=1, encoding='utf-8')

        # Check if DataFrame is empty
        if df.empty:
            print(f"File {file} is empty or unreadable")
            continue

        # Filtering based on 'stop_codon' and calculating CDR lengths
        if 'stop_codon' in df.columns:
            filtered_data = df[df['stop_codon'] == 'F']

            # Calculate CDR lengths if necessary columns are present
            if all(col in df.columns for col in ['cdr1_start', 'cdr1_end', 'cdr2_start', 'cdr2_end', 'cdr3_start', 'cdr3_end']):
                filtered_data['cdr1_length'] = filtered_data['cdr1_end'] - filtered_data['cdr1_start'] + 1
                filtered_data['cdr2_length'] = filtered_data['cdr2_end'] - filtered_data['cdr2_start'] + 1
                filtered_data['cdr3_length'] = filtered_data['cdr3_end'] - filtered_data['cdr3_start'] + 1

                # Filter based on CDR positions and lengths
                filtered_data = filtered_data[
                    (filtered_data['cdr1_start'] >= 20) &
                    (filtered_data['sequence'].str.len() - filtered_data['cdr3_end'] >= 10) &
                    (filtered_data['cdr1_length'].between(5, 12)) &
                    (filtered_data['cdr2_length'].between(1, 10)) &
                    (filtered_data['cdr3_length'].between(5, 38))
                ]

                # Extract amino acid sequences and append to all_sequences
                all_sequences.extend(filtered_data['sequence'].dropna().tolist())
            else:
                print(f"File {file} is missing necessary CDR columns.")
        else:
            print(f"File {file} is missing 'stop_codon' column.")

    except Exception as e:
        print(f"Error reading file {file}: {e}")

# 4. Split into train, validation, and test sets
train_seqs, temp_seqs = train_test_split(all_sequences, test_size=0.2, random_state=42)
val_seqs, test_seqs = train_test_split(temp_seqs, test_size=0.5, random_state=42)

# 5. Save to .txt files
for seqs, filename in zip([train_seqs, val_seqs, test_seqs], ['train.txt', 'val.txt', 'test.txt']):
    with open(filename, 'w') as f:
        for seq in seqs:
            f.write(seq + '\n')  # Only writing sequences, not additional columns

# Print the number of sequences remaining after filtering
print(f"Number of sequences after filtering: {len(all_sequences)}")


Number of sequences after filtering: 3297


In [ ]:
# Cách đọc file bằng Python tiêu chuẩn
with open("./assets/train.txt", 'r') as f:
    train_txt = f.readlines()

# Loại bỏ ký tự xuống dòng "\n" ở cuối mỗi dòng
train_txt = [line.strip() for line in train_txt]

# Xem dữ liệu
print(train_txt[:5])  # Xem 5 dòng đầu tiên

# Kiểm tra số lượng dòng trong file train.txt
num_lines = len(train_txt)
print(f"Number of rows in the file train.txt: {num_lines}")


['GGGGGTCTCAGGAGGCAGCGCTCTCGGGACGTCTCCACCATGGCCTGGGCTCTGCTATTCCTCACCCTCCTCACTCAGGGCACAGGGTCCTGGGCCCAGTCTGCCCTGACTCAGCCTGCCTCCGTGTCTGGGTCTCCTGGACAGTCGATCACCATCTCCTGCACTGGAACCAGCAACTATGTCTCCTGGTACCAACAACACCCCGGCAAAGCCCCCAAACTCATGATTTATGATGTCACTAATCGGCCCTCAGGGATTTCTAATCGCTTCTCTGCCTCCAAGTCTGGCAACACGGCCTCCCTGACCATCTCTGGGCTCCAGGCTGAGGACGAGGCTGATTACTACTGCAGTTCATATACAATTGGCAGAGCTGTCTTCGGAAGTGGGACCAAGGTCACCGTCCTAGGTCAGCCCAAGGCCAACCCCACTGTCACTCTGTTCCCGCCCTCCTCTGAGGAGCTACAAGCCAACGT', 'GCTCACTGCACAGGGTCCTGGGCCCAGTCTGTATTGACGCAGCCGCCTTCAGTGTCTGCGGCCCCCGGACAAAGGGTCACCATCTCCTGCTTTGGAAGCAGATTCAGTTCTGTCTCCTGGTACCAACAACTCCCAGGGACAGCCCCCAAACTCCTCATCTATCCAAATGGTGAGCGGCCCTCAGGCATTCCTGACCGATTTTTTGGCTCCGAGTCTGGCACGTCAGCCACCCTGGGCATCACCGCAGTCCAGACTGGGGACGAGGCCGATTATTATTGCGGAACATGGGATGACAGCCTGAGTTCTGTGGTCTTCGGCGGAGGACCAAGCTCGACCGTACCTAAGTCACGCCCGAAGGCTCGCCCCCTCGGTCACTCTCGTTCCCGCCCTCTCTAGTGGAGTCTTACAACGCCAAACGAACGGCCCAACACGTGGTCG', 'GGGGGTCACAAGAGGCAGCGCTCTCGGGACGTCTCCACCATGGCCTGGGCTCTGCTGCTCCTCACTCTCCTCACTCAGG

In [ ]:
text_datasets = {
    "train": ['./assets/train.txt'],
    "eval": ['./assets/val.txt'],
    "test": ['./assets/test.txt']
}

dataset = load_dataset("text", data_files=text_datasets)
tokenized_dataset = dataset.map(
    lambda z: tokenizer(
        z["text"],
        padding="max_length",
        truncation=True,
        max_length=150,
        return_special_tokens_mask=True,
    ),
    batched=True,
    num_proc=1,
    remove_columns=["text"],
)

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/2637 [00:00<?, ? examples/s]

Map:   0%|          | 0/330 [00:00<?, ? examples/s]

Map:   0%|          | 0/330 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset['train']

Dataset({
    features: ['input_ids', 'token_type_ids', 'special_tokens_mask', 'attention_mask'],
    num_rows: 2637
})

## Model configuration

In [5]:
# These are the cofigurations we've used for pre-training.
antiberta_config = {
    "num_hidden_layers": 12,
    "num_attention_heads": 12,
    "hidden_size": 768,
    "d_ff": 3072,
    "vocab_size": 25,
    "max_len": 150,
    "max_position_embeddings": 152,
    "batch_size": 32,
    "max_steps": 5000,
    "weight_decay": 0.01,
    "peak_learning_rate": 0.0001,
}


In [6]:
model_config = RobertaConfig(
    vocab_size=antiberta_config.get("vocab_size"),
    hidden_size=antiberta_config.get("hidden_size"),
    max_position_embeddings=antiberta_config.get("max_position_embeddings"),
    num_hidden_layers=antiberta_config.get("num_hidden_layers", 12),
    num_attention_heads=antiberta_config.get("num_attention_heads", 12),
    type_vocab_size=1,
)
model = RobertaForMaskedLM(model_config)

In [ ]:
#training arguments
args = TrainingArguments(
    output_dir="test5", ##change your output directory to save checkpoint
    overwrite_output_dir=True,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    max_steps=5000,
    save_steps=200,
    logging_steps=200,
    adam_beta2=0.98,
    adam_epsilon=1e-6,
    weight_decay=0.01,
    warmup_steps=100,
    learning_rate=1e-4,
    gradient_accumulation_steps=1,
    fp16=True,
    evaluation_strategy="steps",
    seed=42
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


## Setup of the HuggingFace Trainer

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    data_collator=collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["eval"]
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()

Step,Training Loss,Validation Loss
200,1.020900,0.421411
400,0.280900,0.222352
600,0.183500,0.175487
800,0.149000,0.145062
1000,0.136500,0.135046
1200,0.122100,0.134074
1400,0.117300,0.118166
1600,0.111900,0.114940
1800,0.104200,0.119441
2000,0.101200,0.113656


TrainOutput(global_step=5000, training_loss=0.13766126956939698, metrics={'train_runtime': 772.5428, 'train_samples_per_second': 207.108, 'train_steps_per_second': 6.472, 'total_flos': 2.4165821771208e+16, 'train_loss': 0.13766126956939698, 'epoch': 60.24096385542169})

In [ ]:
trainer.save_model(options.dir)

In [ ]:
out = trainer.predict(tokenized_dataset['test'])